## Analysing a suitable location for a new restaurant near Manhattan

References: Coursera Applied Data Science Capstone course material for week 3.

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Open the json file containing New York data

In [21]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']

Transform the data in nested Python dictionaries into a *pandas* dataframe. 

In [3]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names) #instantiating the data frame

#looping through the data, filling the dataframe...
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    

Slicing the original dataframe and creating a new dataframe of the Manhattan data.

In [20]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head(7)

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211


Obtaining the geographical coordinates of Manhattan.

In [5]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


Visualizing the neighbourhoods of Manhattan.

In [6]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Defining Foursquare Credentials and Version

In [7]:
CLIENT_ID = 'LPACR0WWYLDOVEC05VC3NO22KKOJHSYMPXDL4UCKL5NFO2KX'
CLIENT_SECRET = '3OWIU3WIELJZINL2ZFKXIANQLJA1TA1LGDGXNACDGOIM5ADG'
VERSION = '20180605' # Foursquare API version

Defining and calling a function to analyse the neighbourhoods of Manhattan:
(to get the top 50 venues that are in Marble Hill within a radius of 500 meters)

In [8]:
LIMIT = 50
    
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Grouping the neighborhoods:

In [9]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

Function to sort the venues in descending order.

In [10]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Defining new dataframe and displaying the top 5 venues for each neighborhood.

In [11]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Memorial Site,BBQ Joint,Food Court
1,Carnegie Hill,Pizza Place,Bookstore,Coffee Shop,Spa,Italian Restaurant
2,Central Harlem,African Restaurant,Gym / Fitness Center,American Restaurant,Bar,Public Art
3,Chelsea,Ice Cream Shop,Hotel,Nightclub,Cupcake Shop,Seafood Restaurant
4,Chinatown,Chinese Restaurant,Salon / Barbershop,Spa,Sandwich Place,Cocktail Bar


Run *k*-means to cluster the neighborhood into 5 clusters.

In [12]:
# set number of clusters
kclusters = 4

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 2, 2, 2, 1, 3, 2, 2])

Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [13]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


Visualizing the resulting clusters

In [14]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)
    
       
map_clusters

Examining cluster 1:

In [15]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Marble Hill,Sandwich Place,Coffee Shop,Yoga Studio,Clothing Store,Seafood Restaurant
11,Roosevelt Island,Park,Deli / Bodega,Coffee Shop,Sandwich Place,Noodle House
26,Morningside Heights,Bookstore,American Restaurant,Coffee Shop,Park,Tennis Court
28,Battery Park City,Park,Coffee Shop,Memorial Site,BBQ Joint,Food Court
37,Stuyvesant Town,Park,Boat or Ferry,Bar,Playground,Cocktail Bar


Examining cluster 2:

In [16]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,Lincoln Square,Theater,Concert Hall,Plaza,Indie Movie Theater,Opera House
14,Clinton,Theater,Gym / Fitness Center,Wine Shop,Indie Theater,Lounge


Examining cluster 3:

In [17]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Chinatown,Chinese Restaurant,Salon / Barbershop,Spa,Sandwich Place,Cocktail Bar
6,Central Harlem,African Restaurant,Gym / Fitness Center,American Restaurant,Bar,Public Art
8,Upper East Side,Italian Restaurant,Hotel,Boutique,Bakery,American Restaurant
9,Yorkville,Italian Restaurant,Gym,Deli / Bodega,Coffee Shop,Café
10,Lenox Hill,Burger Joint,Gym,Gym / Fitness Center,Bakery,Turkish Restaurant
12,Upper West Side,Seafood Restaurant,Bakery,Cosmetics Shop,Italian Restaurant,Bar
15,Midtown,Clothing Store,Sporting Goods Shop,Bookstore,Cocktail Bar,Coffee Shop
16,Murray Hill,Hotel,Japanese Restaurant,Coffee Shop,Burger Joint,Italian Restaurant
17,Chelsea,Ice Cream Shop,Hotel,Nightclub,Cupcake Shop,Seafood Restaurant
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,Café,Cosmetics Shop,Clothing Store


Examining cluster 4:

In [18]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Washington Heights,Café,Coffee Shop,Mobile Phone Shop,Wine Shop,Bakery
3,Inwood,Café,Mexican Restaurant,Pizza Place,American Restaurant,Park
4,Hamilton Heights,Café,Mexican Restaurant,Coffee Shop,Yoga Studio,Sushi Restaurant
5,Manhattanville,Coffee Shop,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Sushi Restaurant
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Thai Restaurant
25,Manhattan Valley,Indian Restaurant,Coffee Shop,Pizza Place,Playground,Bar
36,Tudor City,Park,Deli / Bodega,Mexican Restaurant,Café,Thai Restaurant
